# PYNQ Board Facial Recognition Test

This notebook provides a simple test of the facial recognition system on the PYNQ board.
It includes setup, face detection, and recognition from a camera feed.

## 1. Install Required Dependencies

First, we need to install the necessary packages. This may take some time on the PYNQ board.

In [ ]:
# Install required packages (only run once)
# Import sys to check if we're running the correct Python version
import sys
print(f"Python version: {sys.version}")

# Uncomment and run this cell if you need to install dependencies
# !pip install numpy opencv-python pillow torch==1.13.1 torchvision==0.14.1 facenet-pytorch --index-url https://download.pytorch.org/whl/cpu

## 2. Import Libraries

Let's import the necessary libraries for the facial recognition system.

In [ ]:
import os
import cv2
import numpy as np
import pickle
import time
import torch
from PIL import Image

# Check if GPU is available (it won't be on PYNQ board)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 3. Load Face Recognition Models

Now we'll load the MTCNN model for face detection and InceptionResnetV1 for face recognition.

In [ ]:
# Import facenet-pytorch models
try:
    from facenet_pytorch import MTCNN, InceptionResnetV1
    print("Successfully imported facenet-pytorch models")
    
    # Initialize models with lightweight settings for PYNQ
    mtcnn = MTCNN(
        image_size=160,
        margin=0,
        min_face_size=40,
        thresholds=[0.6, 0.7, 0.9],
        factor=0.709,
        keep_all=True,
        device=device
    )
    
    resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)
    print("Models loaded successfully")
    
except Exception as e:
    print(f"Error loading models: {e}")
    print("Please make sure facenet-pytorch is installed")

## 4. Define Helper Functions

Let's define the helper functions for loading known faces and comparing them with detected faces.

In [ ]:
def load_known_faces(pickle_path):
    """Load known face embeddings and names from a pickle file."""
    if os.path.exists(pickle_path):
        try:
            with open(pickle_path, 'rb') as f:
                data = pickle.load(f)
            return np.array(data['embeddings']), data['names']
        except Exception as e:
            print(f"Error loading known faces: {e}")
            return np.array([]), []
    else:
        print(f"Warning: No known faces file at {pickle_path}")
        return np.array([]), []

def compare_faces(face_embedding, known_embeddings, known_names, threshold=0.9):
    """Compare a face embedding with known embeddings."""
    if known_embeddings.size == 0:
        return "Unknown", 1.0
    
    # Compute Euclidean distances
    distances = np.linalg.norm(known_embeddings - face_embedding, axis=1)
    min_index = np.argmin(distances)
    min_distance = distances[min_index]
    
    if min_distance < threshold:
        return known_names[min_index], min_distance
    else:
        return "Unknown", min_distance

## 5. Load Known Faces

Let's load the pre-trained face embeddings from the pickle file.

In [ ]:
# Path to known faces pickle file - using current notebook directory by default
# If your pickle file is somewhere else on the PYNQ board, update this path
current_dir = os.path.dirname(os.path.abspath('__file__'))
PICKLE_PATH = os.path.join(current_dir, 'known_faces.pkl')

# If file doesn't exist in current directory, try the data directory
if not os.path.exists(PICKLE_PATH):
    PICKLE_PATH = os.path.join(current_dir, 'data', 'known_faces.pkl')

print(f"Looking for known faces file at: {PICKLE_PATH}")

# Load known faces
known_embeddings, known_names = load_known_faces(PICKLE_PATH)
print(f"Loaded {len(known_names)} known faces")

# Print the names of known people
if known_names:
    print("Known identities:")
    for name in set(known_names):
        print(f"- {name}")

## 6. Test Camera Connection

Let's check if we can connect to the camera.

In [ ]:
def test_camera(camera_index=0):
    """Test if we can connect to the camera at the specified index."""
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print(f"Could not open camera at index {camera_index}")
        return False
    
    ret, frame = cap.read()
    if not ret:
        print("Could not read frame from camera")
        cap.release()
        return False
    
    print(f"Successfully connected to camera at index {camera_index}")
    print(f"Frame dimensions: {frame.shape[1]}x{frame.shape[0]}")
    cap.release()
    return True

# Try camera indices 0 and 1
camera_index = 0
if not test_camera(camera_index):
    camera_index = 1
    test_camera(camera_index)

## 7. Capture and Process a Single Frame

Let's capture a single frame from the camera and process it to detect faces.

In [ ]:
# Capture a single frame and process it
def process_single_frame(camera_idx=0):
    cap = cv2.VideoCapture(camera_idx)
    if not cap.isOpened():
        print(f"Could not open camera at index {camera_idx}")
        return
    
    # Set lower resolution for better performance
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    # Capture frame
    ret, frame = cap.read()
    if not ret:
        print("Could not read frame from camera")
        cap.release()
        return
    
    # Convert BGR to RGB (OpenCV uses BGR, PyTorch uses RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Detect faces
    start_time = time.time()
    boxes, _ = mtcnn.detect(rgb_frame)
    
    # Process each detected face
    if boxes is not None:
        print(f"Detected {len(boxes)} face(s)")
        
        for box in boxes:
            # Get coordinates
            x1, y1, x2, y2 = [int(b) for b in box]
            
            # Make sure box coordinates are within frame
            x1, y1 = max(0, x1), max(0, y1)
            x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
            
            if x2 > x1 and y2 > y1:  # Valid box
                # Extract face
                face_img = rgb_frame[y1:y2, x1:x2]
                
                # Convert to PIL and process
                pil_img = Image.fromarray(face_img).resize((160, 160))
                
                # Get embedding
                face_tensor = mtcnn(pil_img).unsqueeze(0).to(device)
                if face_tensor is not None:
                    with torch.no_grad():
                        embedding = resnet(face_tensor).cpu().numpy().flatten()
                    
                    # Compare with known faces
                    name, min_dist = compare_faces(embedding, known_embeddings, known_names)
                    print(f"Recognized: {name} (confidence: {1-min_dist:.2f})")
                    
                    # Draw rectangle and name
                    color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, f"{name} ({min_dist:.2f})", (x1, y1 - 10),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    else:
        print("No faces detected")
    
    end_time = time.time()
    processing_time = end_time - start_time
    print(f"Processing time: {processing_time:.2f} seconds")
    
    cap.release()
    
    # Convert BGR to RGB for display in notebook
    result_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    return result_rgb

# Process a single frame from the camera
result = process_single_frame(camera_index)

# Display the result
if result is not None:
    from IPython.display import display
    from PIL import Image
    display(Image.fromarray(result))

## 8. Live Video Processing (Use with Caution)

This cell processes live video for a short duration. It might be resource-intensive on the PYNQ board, so use it with caution.

In [ ]:
# Note: This cell might be resource-intensive for the PYNQ board
# It stops after a few seconds to prevent overloading

def process_video_briefly(camera_idx=0, duration=5, skip_frames=3):
    """Process video for a short duration, skipping frames for better performance.
    
    Args:
        camera_idx: Camera index (0 or 1)
        duration: Duration in seconds to run
        skip_frames: Process every Nth frame for better performance
    """
    # Create and configure VideoCapture
    cap = cv2.VideoCapture(camera_idx)
    if not cap.isOpened():
        print(f"Could not open camera at index {camera_idx}")
        return
    
    # Set resolution for better performance
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)  # Lower resolution for PYNQ
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)  # Lower resolution for PYNQ
    
    # Create output for video display in notebook
    from IPython.display import display, clear_output
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 6))
    
    # Process frames for specified duration
    start_time = time.time()
    frame_count = 0
    try:
        while (time.time() - start_time) < duration:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            if frame_count % skip_frames != 0:
                continue  # Skip this frame
            
            # Convert to RGB for processing
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # Detect faces
            try:
                boxes, _ = mtcnn.detect(rgb_frame)
                
                if boxes is not None:
                    for box in boxes:
                        # Get coordinates
                        x1, y1, x2, y2 = [int(b) for b in box]
                        
                        # Make sure coordinates are valid
                        x1, y1 = max(0, x1), max(0, y1)
                        x2, y2 = min(frame.shape[1], x2), min(frame.shape[0], y2)
                        
                        if x2 > x1 and y2 > y1:
                            # Extract face
                            face_img = rgb_frame[y1:y2, x1:x2]
                            
                            # Convert to PIL and get embedding
                            pil_img = Image.fromarray(face_img).resize((160, 160))
                            face_tensor = mtcnn(pil_img).unsqueeze(0).to(device)
                            
                            if face_tensor is not None:
                                with torch.no_grad():
                                    embedding = resnet(face_tensor).cpu().numpy().flatten()
                                
                                # Compare with known faces
                                name, min_dist = compare_faces(embedding, known_embeddings, known_names)
                                
                                # Draw rectangle and name
                                color = (0, 255, 0) if name != "Unknown" else (255, 0, 0)
                                cv2.rectangle(frame, (x1, y1), (x2, y2), color[::-1], 2)  # BGR format for OpenCV
                                cv2.putText(frame, f"{name} ({min_dist:.2f})", (x1, y1 - 10),
                                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color[::-1], 2)
            except Exception as e:
                print(f"Error in frame processing: {e}")
            
            # Display the result (clear previous output and show new frame)
            clear_output(wait=True)
            plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            plt.axis('off')
            display(plt.gcf())
            plt.pause(0.01)
            
    except Exception as e:
        print(f"Error in video processing: {e}")
    finally:
        cap.release()
        print(f"Processed {frame_count // skip_frames} frames over {duration} seconds")

# Uncomment to run live video processing (use with caution on PYNQ board)
# process_video_briefly(camera_index, duration=5, skip_frames=5)

## 9. Performance Monitoring

Let's test the performance of different components to understand the bottlenecks on the PYNQ board.

In [ ]:
def performance_test():
    """Test the performance of different components."""
    print("Running performance tests...")
    
    # Test face detection
    cap = cv2.VideoCapture(camera_index)
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("Could not read frame from camera")
        return
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Test MTCNN detection performance
    start_time = time.time()
    boxes, _ = mtcnn.detect(rgb_frame)
    mtcnn_time = time.time() - start_time
    print(f"MTCNN face detection: {mtcnn_time:.4f} seconds")
    
    # Skip face recognition if no faces detected
    if boxes is None or len(boxes) == 0:
        print("No faces detected for recognition test")
        return
    
    # Extract a face for recognition test
    x1, y1, x2, y2 = [int(b) for b in boxes[0]]
    face_img = rgb_frame[y1:y2, x1:x2]
    pil_img = Image.fromarray(face_img).resize((160, 160))
    
    # Test face embedding generation
    start_time = time.time()
    face_tensor = mtcnn(pil_img).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = resnet(face_tensor)
    embedding_time = time.time() - start_time
    print(f"Face embedding generation: {embedding_time:.4f} seconds")
    
    # Test face comparison
    start_time = time.time()
    compare_faces(embedding.cpu().numpy().flatten(), known_embeddings, known_names)
    comparison_time = time.time() - start_time
    print(f"Face comparison: {comparison_time:.4f} seconds")
    
    # Total time for one face processing
    total_time = mtcnn_time + embedding_time + comparison_time
    print(f"Total processing time for one face: {total_time:.4f} seconds")
    print(f"Theoretical max FPS: {1/total_time:.2f}")
    
    # Memory usage
    try:
        import psutil
        process = psutil.Process(os.getpid())
        memory_info = process.memory_info()
        print(f"Memory usage: {memory_info.rss / (1024 * 1024):.2f} MB")
    except ImportError:
        print("psutil not available, skipping memory usage info")

# Run performance tests
try:
    performance_test()
except Exception as e:
    print(f"Error during performance test: {e}")

## 10. Cleanup

Let's clean up to release resources.

In [ ]:
# Clean up resources
import gc

# Delete large objects
del mtcnn
del resnet
del known_embeddings

# Run garbage collector
gc.collect()

# If using CUDA (not on PYNQ)
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Resources cleaned up")